# Etapa 2
- Calculo da Entropia e métricas.
- Criação do agente de reforço para seleção da próxima rotulação.

In [1]:
from tqdm import tqdm
import numpy as np
from tqdm import tqdm
from pathlib import Path
from ultralytics import YOLO
from ultralytics.engine.results import Results
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import cv2

In [2]:
model = YOLO("runs/detect/yolov11-initial/weights/best.pt") 

In [3]:

metrics = model.val(
    data="coco.yaml",
    split="val",
    imgsz=480,
    conf=0.5,  # Limiar de confiança
    iou=0.6, # Limiar de NMS
)


Ultralytics 8.3.135  Python-3.13.2 torch-2.7.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 8192MiB)
YOLO11x summary (fused): 190 layers, 56,919,424 parameters, 0 gradients, 194.9 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1562.2977.0 MB/s, size: 160.4 KB)


val: Scanning E:\COCO-Dataset\val2017\val\labels.cache... 4000 images, 39 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 250/250 [02:50<00:00,  1.46it/s]


                   all       4000      29401      0.793      0.519      0.667       0.54
                person       2167       8916      0.935      0.569      0.756      0.629
               bicycle        118        250      0.816      0.516      0.681      0.453
                   car        432       1587      0.876      0.452      0.674      0.528
            motorcycle        132        311      0.837      0.627      0.759      0.577
              airplane         80        117      0.868      0.786      0.868      0.749
                   bus        153        239      0.775       0.72      0.791       0.71
                 train        134        160      0.968      0.756      0.872      0.753
                 truck        198        318      0.503      0.557       0.53      0.429
                  boat         97        341       0.77      0.372      0.575      0.379
         traffic light        146        475      0.782      0.356      0.573      0.387
          fire hydran

In [ ]:
metrics

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
       78, 79])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020B14B6F8C0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0

In [8]:
def calculate_entropy(probabilities):
    """Calcula a entropia de Shannon para um vetor de probabilidades."""
    epsilon = 1e-10  # Evitar log(0)
    return -np.sum(probabilities * np.log(probabilities + epsilon), axis=-1)

In [16]:
class CocoValDataset(Dataset):
    def __init__(self, base_path, img_size=480):
        self.img_dir = Path(base_path) / 'val' / 'images'
        self.img_files = list(self.img_dir.glob('*.jpg'))
        
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((img_size, img_size)),
            transforms.Normalize((0.5,),(0.5,))
        ])

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = self.img_files[idx]
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return self.transform(img), img_path.name



def validate_model(model, device, data_path, batch_size=8):
    dataset = CocoValDataset(data_path)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    model.eval()
    all_confs = []
    all_entropies = []
    
    with torch.no_grad():
        for batch, img_names in tqdm(loader, desc='Validating'):
            batch = batch.to(device)
            
            # Supondo que o modelo retorna detecções no formato YOLO
            results = model(batch, save_conf=True)
            
            # Processar cada imagem no batch
            
            for i, result in enumerate(results):
    
                # Cálculo de confiança e das probabilidades nas bounding boxes
                if hasattr(result, 'boxes') and result.boxes is not None:
                    confs = result.boxes.conf.cpu().numpy()
                    entropy_per_detection = [calculate_entropy(c) for c in confs]
                    all_entropies.extend(entropy_per_detection)
                    all_confs.extend(confs.tolist())
    
    # Calcular métricas
    avg_conf = np.mean(all_confs) if all_confs else 0
    avg_entropy = np.mean(all_entropies) if all_entropies else 0
    
    print(f'\nConfiança Média: {avg_conf:.4f}')
    print(f'Entropia Média: {avg_entropy:.4f}')
    return avg_conf, avg_entropy


model.to('cuda')
    
data_path = 'E:/COCO-Dataset/val2017/'
avg_conf, avg_entropy = validate_model(model, 'cuda', data_path)



Validating:   0%|          | 0/500 [00:00<?, ?it/s]


WARNING torch.Tensor inputs should be normalized 0.0-1.0 but max value is 1.000000238418579. Dividing input by 255.
0: 480x480 (no detections), 49.8ms
1: 480x480 (no detections), 49.8ms
2: 480x480 (no detections), 49.8ms
3: 480x480 (no detections), 49.8ms
4: 480x480 (no detections), 49.8ms
5: 480x480 (no detections), 49.8ms
6: 480x480 (no detections), 49.8ms
7: 480x480 (no detections), 49.8ms
Speed: 0.0ms preprocess, 49.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 480)


Validating:   0%|          | 1/500 [00:00<04:44,  1.75it/s]


0: 480x480 1 person, 1 cell phone, 49.9ms
1: 480x480 2 laptops, 1 mouse, 49.9ms
2: 480x480 1 bowl, 49.9ms
3: 480x480 3 persons, 1 truck, 2 horses, 1 surfboard, 49.9ms
4: 480x480 3 cars, 2 trucks, 49.9ms
5: 480x480 2 persons, 1 bus, 49.9ms
6: 480x480 1 cat, 4 laptops, 49.9ms
7: 480x480 2 airplanes, 1 bird, 49.9ms
Speed: 0.0ms preprocess, 49.9ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 480)


Validating:   0%|          | 2/500 [00:01<04:28,  1.86it/s]

...


WARNING torch.Tensor inputs should be normalized 0.0-1.0 but max value is 1.000000238418579. Dividing input by 255.
0: 480x480 (no detections), 38.7ms
1: 480x480 (no detections), 38.7ms
2: 480x480 (no detections), 38.7ms
3: 480x480 (no detections), 38.7ms
4: 480x480 (no detections), 38.7ms
5: 480x480 (no detections), 38.7ms
6: 480x480 (no detections), 38.7ms
7: 480x480 (no detections), 38.7ms
Speed: 0.0ms preprocess, 38.7ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 480)


Validating: 100%|█████████▉| 499/500 [03:33<00:00,  2.31it/s]


0: 480x480 3 cars, 3 cows, 36.2ms
1: 480x480 1 fire hydrant, 36.2ms
2: 480x480 1 person, 1 skateboard, 36.2ms
3: 480x480 1 person, 2 zebras, 1 giraffe, 36.2ms
4: 480x480 1 person, 1 cell phone, 36.2ms
5: 480x480 1 person, 1 skateboard, 36.2ms
6: 480x480 1 clock, 36.2ms
7: 480x480 1 toilet, 1 clock, 36.2ms
Speed: 0.0ms preprocess, 36.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 480)


Validating: 100%|██████████| 500/500 [03:33<00:00,  2.34it/s]


Confiança Média: 0.5823
Entropia Média: 0.2723


In [17]:
print(avg_conf, avg_entropy)

0.5823175340445955 0.27230942
